# SVM

Este código é o primeiro código para desenvolvermos um modelo. Vamos iniciar com o SVM.

In [ ]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.preprocessing import FunctionTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

directory_path = '/content/drive/MyDrive/paic/csv'
# Função para encontrar todos os arquivos CSV em subpastas

def find_all_csv_files(directory):
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

# Obtém a lista de todos os arquivos CSV
csv_files = find_all_csv_files(directory_path)

# Verifica se arquivos CSV foram encontrados
if not csv_files:
    print("Nenhum arquivo CSV encontrado.")
else:
    print(f"Arquivos CSV encontrados: {csv_files}")


Arquivos CSV encontrados: ['/content/drive/MyDrive/paic/csv/csv_esporte/pagina_101.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_125.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_108.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_10.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_11.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_115.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_117.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_100.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_118.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_122.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_126.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_106.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_105.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_102.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/pagina_12.csv', '/content/drive/MyDrive/paic/csv/csv_esporte/p

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
dataframes = []
# Itera sobre cada arquivo CSV e lê os dados
for file_path in csv_files:
    df = pd.read_csv(file_path)
    dataframes.append(df)


# Concatena todos os DataFrames em um único DataFrame
content = pd.concat(dataframes, ignore_index=True)
content.count()





,0
titulo,7901
data,7922
texto,7815
categoria,7922


In [ ]:
from sklearn.model_selection import StratifiedKFold

# Converter para string e remover valores nulos
df = pd.DataFrame(content)
df['texto'] = content['texto'].astype(str).fillna('')

# Remover linhas onde a coluna 'texto' está vazia (caso haja espaços em branco ou strings vazias)
df = df[df['texto'].str.strip() != '']

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('portuguese'))
lemmatizer = WordNetLemmatizer()

# Função para pré-processamento de texto
def preprocess_text(text):
    if not isinstance(text, str):  # Verificar se o valor é uma string
        return ''
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub(r'^b\s+', '', text)
    text = text.lower()

    # Remover stopwords e lematizar
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

# Aplicar pré-processamento
df['texto'] = df['texto'].apply(preprocess_text)
print(df['texto'])

# Separar os dados em features (X) e labels (y)
X = df['texto']
y = df['categoria']

# Verifique se todos os valores em X são strings
assert all(isinstance(x, str) for x in X), "Nem todos os valores de X são strings."

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar um pipeline que inclua a vetorização TF-IDF e o classificador SVM
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(max_iter=3000))
])

# Definir os parâmetros para o GridSearch
parameters = {
    'tfidf__max_df': [0.6, 0.7, 0.8],

    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],

    'svm__C': [0.001, 0.01, 0.1, 1],

    'svm__kernel': ['rbf'],
    #determina a "largura" da função de kernel,
    'svm__gamma': ['scale', 'auto'],

    'svm__class_weight': ['balanced'],
}

# Usar GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1, scoring='f1_weighted')
grid_search.fit(X_train, y_train)

# Mostrar os melhores parâmetros encontrados
print(f"Melhores parâmetros: {grid_search.best_params_}")

# Treinar o modelo final com os melhores parâmetros
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Avaliar o modelo nos dados de teste
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

# Salvando o modelo para uso futuro (opcional)
import joblib
joblib.dump(best_model, '/content/drive/MyDrive/paic/csv/svm_model1.pkl')


0       neste domingo confronto válido jogo volta terc...
1       ex jogador ex técnico daseleção brasileiradung...
2       técnico renato gaúcho mostrou pouco aliviado a...
3       copa américa 2024 final neste domingo argentin...
4       goleiro cássio voltou gramados quase três me ú...
                              ...                        
7917    weudson ribeiro colaboração uol brasília 21 01...
7918    murilo matias colaboração uol porto alegre 03 ...
7919    uol paulo 05 08 2021 22h09atualizada em05 08 2...
7920    paulo roberto netto uol brasília 10 07 2022 13...
7921    uol brasília paulo 07 11 2022 14h36atualizada ...
Name: texto, Length: 7922, dtype: object
Fitting 5 folds for each of 72 candidates, totalling 360 fits


Uso do modelo que foi persistido

In [ ]:
import joblib
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Carregar o modelo salvo
model_path = '/content/drive/MyDrive/paic/csv/svm_model1.pkl'
model = joblib.load(model_path)

# Função para pré-processamento de texto
def preprocess_text(text):
    stop_words = set(stopwords.words('portuguese'))
    lemmatizer = WordNetLemmatizer()

    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub(r'^b\s+', '', text)
    text = text.lower()

    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

# Função para fazer previsão com base em um novo texto
def predict_gender(text):
    preprocessed_text = preprocess_text(text)
    prediction = model.predict([preprocessed_text])
    return prediction[0]

# Exemplo de uso
novo_texto = "bolsonaro é o presidente do brasil"
genero_previsto = predict_gender(novo_texto)
print(f"O categoria prevista é: {genero_previsto}")

O categoria prevista é: politica
